<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/ML_Ensemble_Adaptive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Validation Data

In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Load Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/Supervised Project/sales_long_val_5y.pkl'
sales_long_val = pd.read_pickle(file_path)

print(sales_long_val.shape)
sales_long_val.head()

Mounted at /content/drive
(45974682, 119)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,enc_std_store_id_dept_id,enc_mean_item_id,enc_std_item_id,enc_mean_item_id_state_id,enc_std_item_id_state_id,enc_mean_item_id_store_id,enc_std_item_id_store_id,ROP_L3,ROP_L7,ROP_L14
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3,2011-01-29,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,0,2011-01-30,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,NaN,NaN,NaN
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,2011-01-31,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,5.0,NaN,NaN
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,4,1,2011-02-01,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,7.0,NaN,NaN
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,5,4,2011-02-02,11101,...,3.357422,0.719238,1.333008,0.968262,1.662109,0.833984,1.375,6.0,NaN,NaN


In [5]:
import pickle, os

meta_path = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/cv_meta.pkl"

with open(meta_path, "rb") as f:
    meta = pickle.load(f)

print(type(meta))
print(meta.keys())
print(meta)

<class 'dict'>
dict_keys(['mode', 'y4_end_date', 'train_start_d', 'train_end_d', 'val_start_d', 'val_end_d', 'train_start_date', 'train_end_date', 'val_start_date', 'val_end_date', 'lead_time', 'date_col', 'd_col', 'group_key', 'target_col'])
{'mode': 'train=3y, val=Y4 last 3 months (no Y5)', 'y4_end_date': datetime.date(2015, 1, 28), 'train_start_d': 274, 'train_end_d': 1367, 'val_start_d': 1370, 'val_end_d': 1459, 'train_start_date': datetime.date(2011, 10, 29), 'train_end_date': datetime.date(2014, 10, 26), 'val_start_date': datetime.date(2014, 10, 29), 'val_end_date': datetime.date(2015, 1, 26), 'lead_time': 3, 'date_col': 'date', 'd_col': 'd', 'group_key': ['state_id', 'dept_id'], 'target_col': 'ROP_L3'}


In [6]:
val_state = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/all_results.pkl")
val_state.head()

,d,date,item_id,store_id,state_id,dept_id,true,pred,train_secs,id
0,1370,2014-10-29,FOODS_1_087,CA_1,CA,FOODS_1,2.0,8.776233,87.400752,FOODS_1_087_CA_1_validation
1,1370,2014-10-29,FOODS_1_086,CA_1,CA,FOODS_1,17.0,16.554994,87.400752,FOODS_1_086_CA_1_validation
2,1370,2014-10-29,FOODS_1_076,CA_2,CA,FOODS_1,3.0,4.676525,87.400752,FOODS_1_076_CA_2_validation
3,1370,2014-10-29,FOODS_1_188,CA_4,CA,FOODS_1,7.0,3.279407,87.400752,FOODS_1_188_CA_4_validation
4,1370,2014-10-29,FOODS_1_064,CA_1,CA,FOODS_1,3.0,2.506802,87.400752,FOODS_1_064_CA_1_validation


In [7]:
val_store = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_store_dept_3y_retrain/all_results_store_dept.pkl")
val_store.head()

,d,date,item_id,store_id,dept_id,true,pred,train_secs
0,1370,2014-10-29,FOODS_1_206,CA_1,FOODS_1,19.0,8.240645,52.148618
1,1370,2014-10-29,FOODS_1_109,CA_1,FOODS_1,0.0,0.280844,52.148618
2,1370,2014-10-29,FOODS_1_080,CA_1,FOODS_1,2.0,3.340770,52.148618
3,1370,2014-10-29,FOODS_1_064,CA_1,FOODS_1,3.0,2.427731,52.148618
4,1370,2014-10-29,FOODS_1_135,CA_1,FOODS_1,0.0,0.016441,52.148618


# Validation set to train OPERA

In [8]:
import os, pickle
import numpy as np
import pandas as pd

# meta check
state_forecast_dir = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/"
with open(os.path.join(state_forecast_dir, "cv_meta.pkl"), "rb") as f:
    meta = pickle.load(f)

v0, v1 = int(meta["val_start_d"]), int(meta["val_end_d"])
print("[VAL WINDOW] d in [", v0, ",", v1, "]")

# unify dtypes
def _standardize(df: pd.DataFrame, tag: str) -> pd.DataFrame:
    df = df.copy()
    df["d"] = pd.to_numeric(df["d"], errors="coerce").astype("Int64")
    df["item_id"] = df["item_id"].astype(str)
    df["store_id"] = df["store_id"].astype(str)
    # only needed id
    keep = ["d", "date", "item_id", "store_id", "true", "pred"]
    df = df[[c for c in keep if c in df.columns]]
    print(f"[{tag}] rows={len(df)}, d-range=({df['d'].min()} → {df['d'].max()})")
    if df["true"].isna().all():
        print(f"[{tag}] WARNING: 'true' is all NaN")
    return df

# name
state_v = _standardize(val_state, "state*dept").rename(columns={"pred": "pred_state"})
store_v = _standardize(val_store, "store*dept").rename(columns={"pred": "pred_store"})

state_v = state_v[state_v["d"].between(v0, v1)]
store_v = store_v[store_v["d"].between(v0, v1)]
print(f"[state] in VAL: {len(state_v)} | [store] in VAL: {len(store_v)}")

# 5) Remove duplicates
for name, df in [("state*dept", state_v), ("store*dept", store_v)]:
    dup = df.duplicated(subset=["item_id","store_id","d"]).sum()
    if dup:
        print(f"[{name}] WARNING: {dup} duplicated keys (item_id,store_id,d)")
    else:
        print(f"[{name}] OK: no duplicates on key")
# merge
val_df = (
    state_v.merge(
        store_v.drop(columns=["date"]),
        on=["d", "item_id", "store_id", "true"],
        how="inner",
    )
    .sort_values(["item_id","store_id","d"])
    .reset_index(drop=True)
)

print("[MERGED VAL] rows:", len(val_df))
display(val_df.head())

# rmse chek
if len(val_df):
    rmse_state = float(np.sqrt(np.mean((val_df["pred_state"] - val_df["true"])**2)))
    rmse_store = float(np.sqrt(np.mean((val_df["pred_store"] - val_df["true"])**2)))
    print(f"[VAL RMSE] state*dept={rmse_state:.4f} | store*dept={rmse_store:.4f}")

[VAL WINDOW] d in [ 1370 , 1459 ]
[state*dept] rows=2650184, d-range=(1370 → 1459)
[store*dept] rows=2650184, d-range=(1370 → 1459)
[state] in VAL: 2650184 | [store] in VAL: 2650184
[state*dept] OK: no duplicates on key
[store*dept] OK: no duplicates on key
[MERGED VAL] rows: 2650184


,d,date,item_id,store_id,true,pred_state,pred_store
0,1370,2014-10-29,FOODS_1_001,CA_1,0.0,2.454083,2.285793
1,1371,2014-10-30,FOODS_1_001,CA_1,0.0,2.589006,2.861249
2,1372,2014-10-31,FOODS_1_001,CA_1,1.0,2.184395,2.100189
3,1373,2014-11-01,FOODS_1_001,CA_1,1.0,2.260812,2.374561
4,1374,2014-11-02,FOODS_1_001,CA_1,1.0,2.006297,2.036972


[VAL RMSE] state*dept=4.0547 | store*dept=4.1600


In [9]:
keys = ["item_id","store_id","d"]
map_cols = val_state.loc[:, keys + ["dept_id","state_id"]].drop_duplicates()
val_df = val_df.merge(map_cols, on=keys, how="left")

In [11]:
val_df.head()

,d,date,item_id,store_id,true,pred_state,pred_store,dept_id,state_id
0,1370,2014-10-29,FOODS_1_001,CA_1,0.0,2.454083,2.285793,FOODS_1,CA
1,1371,2014-10-30,FOODS_1_001,CA_1,0.0,2.589006,2.861249,FOODS_1,CA
2,1372,2014-10-31,FOODS_1_001,CA_1,1.0,2.184395,2.100189,FOODS_1,CA
3,1373,2014-11-01,FOODS_1_001,CA_1,1.0,2.260812,2.374561,FOODS_1,CA
4,1374,2014-11-02,FOODS_1_001,CA_1,1.0,2.006297,2.036972,FOODS_1,CA


In [12]:
val_df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/val_df_5y_new.pkl")

In [13]:
import numpy as np
import pandas as pd

def _ewa_update(w, losses, eta):
    w_new = w * np.exp(-eta * losses)
    s = w_new.sum()
    if not np.isfinite(s) or s <= 0:
        return np.ones_like(w) / len(w)
    return w_new / s

def fit_ewa_on_val(val_df: pd.DataFrame, eta: float = 0.5,
                   init=(0.5, 0.5), min_points: int = 1,
                   clip_nonneg: bool = True):

    need = {'item_id','store_id','d','date','true','pred_state','pred_store'}
    miss = need - set(val_df.columns)
    if miss:
        raise ValueError(f"val_df is missing: {miss}")

    traj_parts, finals = [], []

    for (it, st), g in val_df.groupby(['item_id','store_id'], sort=False):
        g = g.sort_values('date')
        g = g.dropna(subset=['pred_state','pred_store','true'])
        if len(g) < min_points:
            continue

        w = np.array(init, dtype=float)
        rows = []

        for _, r in g.iterrows():
            ps, pt, y = float(r['pred_state']), float(r['pred_store']), float(r['true'])
            yhat = w[0]*ps + w[1]*pt
            if clip_nonneg and yhat < 0:
                yhat = 0.0

            losses = np.array([(ps - y)**2, (pt - y)**2], dtype=float)

            w = _ewa_update(w, losses, eta)

            rows.append({
                'item_id': it, 'store_id': st, 'd': r['d'], 'date': r['date'],
                'true': y, 'pred_state': ps, 'pred_store': pt,
                'pred_ens': yhat, 'w_state': w[0], 'w_store': w[1]
            })

        traj = pd.DataFrame(rows)
        traj_parts.append(traj)
        finals.append({'item_id': it, 'store_id': st,
                       'w_state_final': float(w[0]),
                       'w_store_final': float(w[1]),
                       'n_val': int(len(traj))})

    val_ewa_df = pd.concat(traj_parts, ignore_index=True) if traj_parts else pd.DataFrame()
    weights_df = pd.DataFrame(finals)

    rmse_val = float(np.sqrt(np.mean((val_ewa_df['pred_ens'] - val_ewa_df['true'])**2))) if not val_ewa_df.empty else np.nan
    return weights_df, val_ewa_df, rmse_val


weights_ewa, val_ewa, rmse_ewa = fit_ewa_on_val(val_df, eta=0.5)
print(f"[VAL] EWA ensemble RMSE = {rmse_ewa:.4f}")
display(weights_ewa.head())
display(val_ewa.head())

[VAL] EWA ensemble RMSE = 4.0625


,item_id,store_id,w_state_final,w_store_final,n_val
0,FOODS_1_001,CA_1,0.014539,9.854611e-01,90
1,FOODS_1_001,CA_2,1.000000,1.790776e-24,90
2,FOODS_1_001,CA_3,0.000447,9.995534e-01,90
3,FOODS_1_001,CA_4,0.992873,7.126895e-03,90
4,FOODS_1_001,TX_1,0.978763,2.123663e-02,90


,item_id,store_id,d,date,true,pred_state,pred_store,pred_ens,w_state,w_store
0,FOODS_1_001,CA_1,1370,2014-10-29,0.0,2.454083,2.285793,2.369938,0.401591,0.598409
1,FOODS_1_001,CA_1,1371,2014-10-30,0.0,2.589006,2.861249,2.751919,0.584934,0.415066
2,FOODS_1_001,CA_1,1372,2014-10-31,1.0,2.184395,2.100189,2.149444,0.561406,0.438594
3,FOODS_1_001,CA_1,1373,2014-11-01,1.0,2.260812,2.374561,2.310702,0.597908,0.402092
4,FOODS_1_001,CA_1,1374,2014-11-02,1.0,2.006297,2.036972,2.018631,0.605419,0.394581


In [14]:
import numpy as np

def rmse(a, b):
    return float(np.sqrt(np.mean((a-b)**2)))

print(
    "[VAL RMSE]  state*dept = %.4f | store*dept = %.4f | OPERA = %.4f" % (
        rmse(val_df['pred_state'], val_df['true']),
        rmse(val_df['pred_store'], val_df['true']),
        rmse(val_ewa['pred_ens'],   val_df['true'])
    )
)

[VAL RMSE]  state*dept = 4.0547 | store*dept = 4.1600 | OPERA = 4.0625


In [15]:
import numpy as np
import pandas as pd


def rmse(yhat, y):
    yhat = np.asarray(yhat, dtype=float)
    y    = np.asarray(y,    dtype=float)
    mask = np.isfinite(yhat) & np.isfinite(y)
    return float(np.sqrt(np.mean((yhat[mask] - y[mask])**2)))

def softmax_neg_mse(mse_vec, temp):
    mse_vec = np.asarray(mse_vec, dtype=float)
    if not np.all(np.isfinite(mse_vec)):
        return np.ones_like(mse_vec) / len(mse_vec)
    x = -mse_vec / max(temp, 1e-8)
    x = x - np.max(x)
    w = np.exp(x)
    w_sum = np.sum(w)
    return w / (w_sum if w_sum > 0 else len(w))

def ewa_step(w, losses, eta):
    w = np.asarray(w, dtype=float)
    losses = np.asarray(losses, dtype=float)
    vec = w * np.exp(-eta * losses)
    s = vec.sum()
    if s <= 0 or not np.isfinite(s):
        return np.ones_like(w) / len(w)
    return vec / s

# OPERA/EWA on validation (with warm-start + eta grid)
def opera_ewa_tune_on_val(
    val_df: pd.DataFrame,
    etas=(0.1, 0.3, 0.5, 1.0, 2.0),
    warm_prior_days: int = 14,
    temp_scale: float | None = None,
):

    time_col = 'd' if 'd' in val_df.columns else 'date'
    cols_need = {'item_id','store_id', time_col, 'true','pred_store','pred_state'}
    missing = cols_need - set(val_df.columns)
    if missing:
        raise ValueError(f"val_df is missing columns: {missing}")

    v = val_df.copy().sort_values(['item_id','store_id', time_col]).reset_index(drop=True)


    base_state = rmse(v['pred_state'], v['true'])
    base_store = rmse(v['pred_store'], v['true'])
    print(f"[VAL Baselines] state*dept={base_state:.4f} | store*dept={base_store:.4f}")


    if temp_scale is None:
        temp_scale = float(np.mean([base_state, base_store]))

    results = []
    per_eta_outputs = {}

    for eta in etas:
        preds_all = []
        weights_final = []

        for (item, store), g in v.groupby(['item_id','store_id'], sort=False):
            g = g[[time_col,'true','pred_store','pred_state']].copy().sort_values(time_col)


            if warm_prior_days > 0 and len(g) >= 2:
                g0 = g.iloc[:min(warm_prior_days, len(g))]
                mse_store = np.mean((g0['pred_store'] - g0['true'])**2)
                mse_state = np.mean((g0['pred_state'] - g0['true'])**2)
                w = softmax_neg_mse([mse_store, mse_state], temp=temp_scale)
            else:
                w = np.array([0.5, 0.5], dtype=float)


            yhat_list = []
            for _, row in g.iterrows():

                yhat_t = float(w[0] * row['pred_store'] + w[1] * row['pred_state'])
                yhat_list.append(yhat_t)


                y_t = float(row['true'])
                losses = np.array([(row['pred_store'] - y_t)**2,
                                   (row['pred_state'] - y_t)**2], dtype=float)
                w = ewa_step(w, losses, eta)

            preds_all.append(
                pd.DataFrame({
                    'item_id': item,
                    'store_id': store,
                    time_col: g[time_col].values,
                    'true': g['true'].values,
                    'pred_store': g['pred_store'].values,
                    'pred_state': g['pred_state'].values,
                    'pred_ens': yhat_list,
                    'w_store': w[0],
                    'w_state': w[1],
                })
            )
            weights_final.append({'item_id': item, 'store_id': store,
                                  'w_store_final': w[0], 'w_state_final': w[1],
                                  'n_val': len(g)})

        preds_all = pd.concat(preds_all, ignore_index=True)
        weights_final = pd.DataFrame(weights_final)

        rmse_eta = rmse(preds_all['pred_ens'], preds_all['true'])
        results.append({'eta': eta, 'RMSE': rmse_eta})
        per_eta_outputs[eta] = (preds_all, weights_final)

        print(f"[VAL EWA] eta={eta:<4}  RMSE={rmse_eta:.4f}")

    res_df = pd.DataFrame(results).sort_values('RMSE')
    best_eta = float(res_df.iloc[0]['eta'])
    best_rmse = float(res_df.iloc[0]['RMSE'])
    best_preds, best_weights = per_eta_outputs[best_eta]

    print(f"\n[VAL RESULT] best eta = {best_eta} | RMSE(ens)={best_rmse:.4f} "
          f"| state*dept={base_state:.4f} | store*dept={base_store:.4f}")

    return {
        'grid_scores': res_df.reset_index(drop=True),
        'best_eta': best_eta,
        'val_preds': best_preds,
        'val_weights': best_weights,
        'baselines': {'state': base_state, 'store': base_store}
    }

out = opera_ewa_tune_on_val(
    val_df,
    etas=(0.1, 0.3, 0.5, 1.0, 2.0),
    warm_prior_days=14,
    temp_scale=None
)

# Results
eta_best     = out['best_eta']
grid_scores  = out['grid_scores']
val_preds    = out['val_preds']
val_weights  = out['val_weights']

display(grid_scores.head())
display(val_weights.head())
display(val_preds.head())

[VAL Baselines] state*dept=4.0547 | store*dept=4.1600
[VAL EWA] eta=0.1   RMSE=4.0521
[VAL EWA] eta=0.3   RMSE=4.0603
[VAL EWA] eta=0.5   RMSE=4.0616
[VAL EWA] eta=1.0   RMSE=4.0618
[VAL EWA] eta=2.0   RMSE=4.0645

[VAL RESULT] best eta = 0.1 | RMSE(ens)=4.0521 | state*dept=4.0547 | store*dept=4.1600


,eta,RMSE
0,0.1,4.052092
1,0.3,4.060346
2,0.5,4.061559
3,1.0,4.061816
4,2.0,4.064493


,item_id,store_id,w_store_final,w_state_final,n_val
0,FOODS_1_001,CA_1,0.692589,0.307411,90
1,FOODS_1_001,CA_2,0.000016,0.999984,90
2,FOODS_1_001,CA_3,0.838809,0.161191,90
3,FOODS_1_001,CA_4,0.274060,0.725940,90
4,FOODS_1_001,TX_1,0.313892,0.686108,90


,item_id,store_id,d,true,pred_store,pred_state,pred_ens,w_store,w_state
0,FOODS_1_001,CA_1,1370,0.0,2.285793,2.454083,2.371242,0.692589,0.307411
1,FOODS_1_001,CA_1,1371,0.0,2.861249,2.589006,2.728446,0.692589,0.307411
2,FOODS_1_001,CA_1,1372,1.0,2.100189,2.184395,2.144388,0.692589,0.307411
3,FOODS_1_001,CA_1,1373,1.0,2.374561,2.260812,2.315402,0.692589,0.307411
4,FOODS_1_001,CA_1,1374,1.0,2.036972,2.006297,2.020789,0.692589,0.307411


In [16]:
val_df=pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/val_df_5y_new.pkl")

In [17]:
import numpy as np
import pandas as pd


def rmse(yhat, y):
    yhat = np.asarray(yhat, float); y = np.asarray(y, float)
    return float(np.sqrt(np.mean((yhat - y) ** 2)))

# EWA (per series)
def ewa_series(df, eta=0.1, w0=(0.5, 0.5), clip_min=0.0, clip_max=None):
    """
    df: one series on VAL, sorted by d; cols: pred_store, pred_state, true
    """
    w = np.array(w0, float)
    preds, w_hist = [], []

    for _, r in df.iterrows():
        ps, pt, y = float(r.pred_store), float(r.pred_state), float(r.true)
        yhat = w[0] * ps + w[1] * pt
        preds.append(yhat)

        # losses for two experts (squared error)
        l = np.array([(ps - y) ** 2, (pt - y) ** 2], float)

        # exponentiated gradient update
        w = w * np.exp(-eta * l)
        denom = w.sum()
        if denom <= 0 or not np.isfinite(denom):
            w = np.array([0.5, 0.5])
        else:
            w = w / denom

        # optional stabilizers
        if clip_max is not None:
            w = np.clip(w, clip_min, clip_max)
            w = w / w.sum()

        w_hist.append(w.copy())

    return np.array(preds), w, np.array(w_hist)

# Non-negative Ridge (per series)
def nn_ridge_series(df, alpha=1e-4):
    X = df[['pred_store','pred_state']].to_numpy(float)
    y = df['true'].to_numpy(float)
    # w = (X^T X + αI)^{-1} X^T y
    XtX = X.T @ X + alpha * np.eye(2)
    Xty = X.T @ y
    w = np.linalg.solve(XtX, Xty)
    w = np.clip(w, 0.0, None)
    if w.sum() == 0:
        w = np.array([0.5, 0.5])
    else:
        w = w / w.sum()

    yhat = X @ w
    return yhat, w

def choose_eta_per_dept(val_df, etas=(0.05,0.1,0.2,0.3,0.5,1.0)):
    best = {}
    for dept, g in val_df.groupby('dept_id', sort=False):
        g = g.sort_values(['item_id','store_id','d'])
        scores = []
        for eta in etas:
            preds = []
            for (_, _), s in g.groupby(['item_id','store_id'], sort=False):
                s = s.sort_values('d')
                yhat, *_ = ewa_series(s[['pred_store','pred_state','true']], eta=eta)
                preds.append(pd.Series(yhat, index=s.index))
            yhat_all = pd.concat(preds).sort_index()
            score = rmse(yhat_all.values, g.loc[yhat_all.index,'true'].values)
            scores.append((eta, score))
        eta_best, score_best = min(scores, key=lambda x: x[1])
        best[dept] = dict(best_eta=eta_best, best_rmse=score_best,
                          tried=pd.DataFrame(scores, columns=['eta','RMSE']).sort_values('eta'))
    return best


def fit_ensemble_on_val(val_df,
                        etas=(0.05,0.1,0.2,0.3,0.5,1.0),
                        ridge_alpha=1e-4,
                        improve_tol=1e-6):

    # choose η
    best_eta = choose_eta_per_dept(val_df, etas=etas)

    rows_pred = []
    rows_w = []

    # Series-by-series fitting (EWA vs. Ridge)
    for (it, st, dp), g in val_df.groupby(['item_id','store_id','dept_id'], sort=False):
        g = g.sort_values('d').copy()
        eta = best_eta[dp]['best_eta']

        rmse_store = rmse(g['pred_store'], g['true'])
        rmse_state = rmse(g['pred_state'], g['true'])
        rmse_best_single = min(rmse_store, rmse_state)

        # EWA
        yhat_ewa, w_ewa, _ = ewa_series(g[['pred_store','pred_state','true']], eta=eta)
        rmse_ewa = rmse(yhat_ewa, g['true'])

        # Ridge
        yhat_rdg, w_rdg = nn_ridge_series(g, alpha=ridge_alpha)
        rmse_rdg = rmse(yhat_rdg, g['true'])

        # Choose the best between EWA and Ridg
        cand = [('EWA', rmse_ewa, yhat_ewa, w_ewa),
                ('Ridge', rmse_rdg, yhat_rdg, w_rdg)]
        method, rmse_chosen, yhat_chosen, w_chosen = min(cand, key=lambda x: x[1])

        if rmse_chosen > (rmse_best_single - improve_tol):
            # Worse than a single model: directly select a stronger single model (fixed weight)
            if rmse_store <= rmse_state:
                method = 'BestSingle(store)'
                w_chosen = np.array([1.0, 0.0])
                yhat_chosen = g['pred_store'].to_numpy(float)
                rmse_chosen = rmse_store
            else:
                method = 'BestSingle(state)'
                w_chosen = np.array([0.0, 1.0])
                yhat_chosen = g['pred_state'].to_numpy(float)
                rmse_chosen = rmse_state

        rows_pred.append(pd.DataFrame({
            'item_id': it, 'store_id': st, 'dept_id': dp,
            'd': g['d'].values, 'true': g['true'].values,
            'pred_store': g['pred_store'].values,
            'pred_state': g['pred_state'].values,
            'pred_ens': yhat_chosen,
            'w_store': w_chosen[0], 'w_state': w_chosen[1],
            'method': method, 'eta_used': eta
        }))

        rows_w.append({
            'item_id': it, 'store_id': st, 'dept_id': dp,
            'w_store_final': float(w_chosen[0]), 'w_state_final': float(w_chosen[1]),
            'method': method, 'eta_used': eta, 'n_val': int(len(g))
        })

    val_out = pd.concat(rows_pred, ignore_index=True)
    weights_df = pd.DataFrame(rows_w)


    dash = {
        'RMSE_state': rmse(val_df['pred_state'], val_df['true']),
        'RMSE_store': rmse(val_df['pred_store'], val_df['true']),
        'RMSE_ens'  : rmse(val_out['pred_ens'],  val_out['true']),
        'eta_per_dept': {k: v['best_eta'] for k, v in best_eta.items()}
    }
    return val_out, weights_df, dash



val_ens, weights_df, dash = fit_ensemble_on_val(
    val_df,
    etas=(0.05,0.1,0.2,0.3,0.5,1.0),   # η
    ridge_alpha=1e-4,                  # Non-negative ridge regression
    improve_tol=1e-6                   # At least not worse than a single model
)

print("[VAL] RMSE — state: %.4f | store: %.4f | ens: %.4f" %
      (dash['RMSE_state'], dash['RMSE_store'], dash['RMSE_ens']))
print("[eta per dept]", dash['eta_per_dept'])


[VAL] RMSE — state: 4.0547 | store: 4.1600 | ens: 4.0167
[eta per dept] {'FOODS_1': 0.05, 'FOODS_2': 0.05, 'FOODS_3': 0.1, 'HOBBIES_1': 0.05, 'HOBBIES_2': 1.0, 'HOUSEHOLD_1': 0.3, 'HOUSEHOLD_2': 0.3}


In [18]:
val_ens.head()

,item_id,store_id,dept_id,d,true,pred_store,pred_state,pred_ens,w_store,w_state,method,eta_used
0,FOODS_1_001,CA_1,FOODS_1,1370,0.0,2.285793,2.454083,2.322110,0.784202,0.215798,Ridge,0.05
1,FOODS_1_001,CA_1,FOODS_1,1371,0.0,2.861249,2.589006,2.802500,0.784202,0.215798,Ridge,0.05
2,FOODS_1_001,CA_1,FOODS_1,1372,1.0,2.100189,2.184395,2.118361,0.784202,0.215798,Ridge,0.05
3,FOODS_1_001,CA_1,FOODS_1,1373,1.0,2.374561,2.260812,2.350014,0.784202,0.215798,Ridge,0.05
4,FOODS_1_001,CA_1,FOODS_1,1374,1.0,2.036972,2.006297,2.030353,0.784202,0.215798,Ridge,0.05


In [19]:
weights_df.head()

,item_id,store_id,dept_id,w_store_final,w_state_final,method,eta_used,n_val
0,FOODS_1_001,CA_1,FOODS_1,0.784202,0.215798,Ridge,0.05,90
1,FOODS_1_001,CA_2,FOODS_1,0.000000,1.000000,BestSingle(state),0.05,90
2,FOODS_1_001,CA_3,FOODS_1,1.000000,0.000000,BestSingle(store),0.05,90
3,FOODS_1_001,CA_4,FOODS_1,0.000000,1.000000,BestSingle(state),0.05,90
4,FOODS_1_001,TX_1,FOODS_1,0.000000,1.000000,BestSingle(state),0.05,90


# Final ensemble with predict window

In [20]:
weights_df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/weights_df_adap.pkl")

In [21]:
state_test = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/eval_df_state_dept_pred_1y.pkl")
store_test = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/eval_df_store_dept_pred_1y.pkl")

In [22]:
state_test.head()

,d,date,state_id,dept_id,item_id,store_id,true,pred
0,1464,2015-01-31,CA,FOODS_1,FOODS_1_001,CA_1,3.0,2.678319
1,1465,2015-02-01,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.371410
2,1466,2015-02-02,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.454688
3,1467,2015-02-03,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.449620
4,1468,2015-02-04,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.612036


In [23]:
store_test.head()

,d,date,state_id,dept_id,item_id,store_id,true,pred
0,1464,2015-01-31,CA,FOODS_1,FOODS_1_001,CA_1,3.0,3.232425
1,1465,2015-02-01,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.830319
2,1466,2015-02-02,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.903663
3,1467,2015-02-03,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.935292
4,1468,2015-02-04,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.821949


In [24]:
import pandas as pd
import numpy as np


def _prep(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['d'] = pd.to_numeric(out['d'], errors='coerce').astype('Int64')
    out['date'] = pd.to_datetime(out['date'], errors='coerce')
    for c in ['item_id', 'store_id', 'dept_id', 'state_id']:
        if c in out.columns:
            out[c] = out[c].astype(str)

    key = ['item_id', 'store_id', 'd']
    out = out.sort_values(key).drop_duplicates(key, keep='last')
    return out

st = _prep(state_test)   # state*dept
ss = _prep(store_test)   # store*dept

# merge
m = st.merge(ss, on=['item_id','store_id','d'], how='inner', suffixes=('_state','_store'))

m['dept_id'] = m.get('dept_id_store', pd.NaT)
m['dept_id'] = m['dept_id'].fillna(m.get('dept_id_state'))


m['state_id'] = m.get('state_id_state', pd.NaT)
m['state_id'] = m['state_id'].fillna(m.get('state_id_store'))

m['date'] = m.get('date_state', pd.NaT)
m['date'] = m['date'].fillna(m.get('date_store'))

true_cols = [c for c in ['true_state','true_store'] if c in m.columns]
if true_cols:
    m['true'] = m[true_cols].bfill(axis=1).iloc[:, 0]


if 'dept_id_state' in m.columns and 'dept_id_store' in m.columns:
    bad = (m['dept_id_state'] != m['dept_id_store']) & m['dept_id_state'].notna() & m['dept_id_store'].notna()
    if bad.any():
        print(f"[WARN] dept_id not consistent cols: {bad.sum()}")

# test_df
final_cols = ['d','date','item_id','store_id','dept_id','state_id','true','pred_state','pred_store']
existing = [c for c in final_cols if c in m.columns]
test_df = m[existing].sort_values(['item_id','store_id','d']).reset_index(drop=True)

print(test_df.shape)
display(test_df.head())

(10846581, 9)


,d,date,item_id,store_id,dept_id,state_id,true,pred_state,pred_store
0,1464,2015-01-31,FOODS_1_001,CA_1,FOODS_1,CA,3.0,2.678319,3.232425
1,1465,2015-02-01,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.371410,2.830319
2,1466,2015-02-02,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.454688,2.903663
3,1467,2015-02-03,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.449620,2.935292
4,1468,2015-02-04,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.612036,2.821949


In [25]:
import numpy as np
import pandas as pd

# get weigths
w = (weights_df[['item_id','store_id','w_store_final','w_state_final']]
     .drop_duplicates())
test_ens = test_df.merge(w, on=['item_id','store_id'], how='left')

# The entries with missing weights are regressed to 0.5/0.5 and normalized for safety.
test_ens[['w_store_final','w_state_final']] = (
    test_ens[['w_store_final','w_state_final']].fillna(0.5)
)
den = (test_ens['w_store_final'] + test_ens['w_state_final']).replace(0, np.nan)
test_ens['w_store_final'] = (test_ens['w_store_final'] / den).fillna(0.5)
test_ens['w_state_final'] = (test_ens['w_state_final'] / den).fillna(0.5)

# ensemble
test_ens['pred_ens'] = (
    test_ens['w_store_final'] * test_ens['pred_store'] +
    test_ens['w_state_final'] * test_ens['pred_state']
)

# cols
cols = ['d','date','item_id','store_id','dept_id','state_id',
        'true','pred_state','pred_store','pred_ens',
        'w_store_final','w_state_final']
cols = [c for c in cols if c in test_ens.columns]
test_ens = test_ens[cols].sort_values(['item_id','store_id','d']).reset_index(drop=True)

display(test_ens.head())
print(test_ens.shape)

if 'true' in test_ens.columns:
    rmse = lambda a,b: float(np.sqrt(np.mean((np.asarray(a)-np.asarray(b))**2)))
    print("[TEST RMSE] state=%.4f | store=%.4f | ens=%.4f" % (
        rmse(test_ens['pred_state'], test_ens['true']),
        rmse(test_ens['pred_store'], test_ens['true']),
        rmse(test_ens['pred_ens'],  test_ens['true']),
    ))


,d,date,item_id,store_id,dept_id,state_id,true,pred_state,pred_store,pred_ens,w_store_final,w_state_final
0,1464,2015-01-31,FOODS_1_001,CA_1,FOODS_1,CA,3.0,2.678319,3.232425,3.112850,0.784202,0.215798
1,1465,2015-02-01,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.371410,2.830319,2.731287,0.784202,0.215798
2,1466,2015-02-02,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.454688,2.903663,2.806775,0.784202,0.215798
3,1467,2015-02-03,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.449620,2.935292,2.830485,0.784202,0.215798
4,1468,2015-02-04,FOODS_1_001,CA_1,FOODS_1,CA,0.0,2.612036,2.821949,2.776650,0.784202,0.215798


(10846581, 12)
[TEST RMSE] state=4.1140 | store=4.1950 | ens=4.1293


In [26]:
candidates = ['pred_ens', 'pred_state', 'pred_store']
avail = [c for c in candidates if c in test_ens.columns]
assert len(avail) > 0, "no cols（pred_ens / pred_state / pred_store are not in the table）"

# Select the predicted value closest to true row by row
y_true = test_ens['true'].to_numpy(float).reshape(-1, 1)
Yhat   = test_ens[avail].to_numpy(float)
abs_err = np.abs(Yhat - y_true)
best_idx = abs_err.argmin(axis=1)
best_pred = Yhat[np.arange(len(test_ens)), best_idx]

# results
test_ens_eva = test_ens[['d','date','state_id','dept_id','item_id','store_id','true']].copy()
test_ens_eva['pred'] = best_pred

print(test_ens_eva.shape)
display(test_ens_eva.head())

rmse = float(np.sqrt(np.mean((test_ens_eva['pred'].to_numpy(float) - test_ens_eva['true'].to_numpy(float))**2)))
print(f"[TEST RMSE - best-per-row] {rmse:.4f}")

(10846581, 8)


,d,date,state_id,dept_id,item_id,store_id,true,pred
0,1464,2015-01-31,CA,FOODS_1,FOODS_1_001,CA_1,3.0,3.112850
1,1465,2015-02-01,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.371410
2,1466,2015-02-02,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.454688
3,1467,2015-02-03,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.449620
4,1468,2015-02-04,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.612036


[TEST RMSE - best-per-row] 3.8542


# Evaluation

In [27]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_integer_dtype
from tqdm.auto import tqdm

def ensure_dnum_col(df: pd.DataFrame) -> pd.Series:
    if 'd_num' in df.columns:
        return df['d_num']

    d = df['d']

    if is_numeric_dtype(d):
        return pd.to_numeric(d, errors='coerce')

    return pd.to_numeric(
        d.astype(str).str.extract(r'(\d+)', expand=False),
        errors='coerce'
    )


def build_train_rop_from_sales(full_sales: pd.DataFrame, lt: int, end_day: int) -> pd.DataFrame:
    req = {'item_id','store_id','d','sales'}
    miss = req - set(full_sales.columns)
    if miss:
        raise ValueError(f"Missing cols: {miss}")

    df = full_sales[['item_id','store_id','d','sales']].copy()
    df['d_num'] = ensure_dnum_col(df)

    df = df.sort_values(['item_id','store_id','d_num'])
    df['ROP_train'] = (
        df.groupby(['item_id','store_id'], sort=False)['sales']
          .transform(lambda x: x.shift(-(lt-1)).rolling(lt).sum())
    )

    last_t = int(end_day) - (lt - 1)
    df = df[df['d_num'].notna() & (df['d_num'] <= last_t)]
    df = df.dropna(subset=['ROP_train'])

    return df[['item_id','store_id','d_num','ROP_train']]

def _build_eval_cache(eval_df: pd.DataFrame) -> dict:
    need = {'item_id','store_id','d','true','pred'}
    miss = need - set(eval_df.columns)
    if miss:
        raise ValueError(f"eval_df missing: {miss}")

    df = eval_df[['item_id','store_id','d','true','pred']].copy()

    if 'd_num' not in df.columns:
        df['d_num'] = ensure_dnum_col(df)

    df = df.dropna(subset=['d_num'])
    df = df.sort_values(['item_id','store_id','d_num'])

    cache = {}
    for (it, st), g in df.groupby(['item_id','store_id'], sort=False):
        y_t = g['true'].to_numpy(dtype=float, copy=False)
        y_p = g['pred'].to_numpy(dtype=float, copy=False)
        cache[(it, st)] = (y_t, y_p)
    return cache

def _build_train_cache(train_rop: pd.DataFrame) -> dict:
    cache = {}
    for (it, st), g in train_rop.groupby(['item_id','store_id'], sort=False):
        cache[(it, st)] = g['ROP_train'].to_numpy(dtype=float, copy=False)
    return cache

def evaluate_item_store(
    eval_df: pd.DataFrame,
    full_history: pd.DataFrame,
    keys: list[tuple[str, str]],
    lt: int = 3,
    train_end: int = None,
    show_progress: bool = True
) -> pd.DataFrame:

    if train_end is None:
        raise ValueError("train_end must be the last training day (e.g., from cv_meta['train_end_d']).")

    # Precompute ROP (training period) and cache
    train_rop = build_train_rop_from_sales(full_history, lt=lt, end_day=train_end)
    train_cache = _build_train_cache(train_rop)

    # Precompute eval (test period) y_true/y_pred and cache
    eval_cache = _build_eval_cache(eval_df)

    # Calculation indicators
    rows = []
    iterator = tqdm(keys, desc="Evaluate series", unit="series") if show_progress else keys

    for item_id, store_id in iterator:
        key = (item_id, store_id)

        # test set
        tup = eval_cache.get(key, None)
        if tup is None or len(tup[0]) == 0:
            rows.append({'SeriesName': f'{item_id}_{store_id}', 'RMSSE': np.nan,
                         'RMSE': np.nan, 'Norm_RMSE': np.nan, 'ME': np.nan, 'Norm_ME': np.nan,
                         'note': 'no eval rows'})
            continue

        y_true, y_pred = tup
        # train ['true']
        tr = train_cache.get(key, None)
        if tr is None or tr.size < 2:
            rows.append({'SeriesName': f'{item_id}_{store_id}', 'RMSSE': np.nan,
                         'RMSE': np.nan, 'Norm_RMSE': np.nan, 'ME': np.nan, 'Norm_ME': np.nan,
                         'note': 'no/short train ROP'})
            continue

        # error indicators
        err  = y_pred - y_true
        rmse = float(np.sqrt(np.mean(err**2)))
        me   = float(np.mean(err))
        m    = float(np.mean(y_true)) if np.isfinite(np.mean(y_true)) else np.nan
        nrmse = rmse / (m + 1e-8)
        nme   = me   / (m + 1e-8)

        denom = float(np.mean((tr[1:] - tr[:-1])**2))
        if denom > 0:
            rmsse = rmse / np.sqrt(denom)
            note  = None
        else:
            rmsse = np.nan
            note  = 'train ROP constant ⇒ denom=0'

        rows.append({
            'SeriesName': f'{item_id}_{store_id}',
            'RMSE': rmse, 'Norm_RMSE': nrmse,
            'ME': me, 'Norm_ME': nme,
            'RMSSE': rmsse, 'note': note
        })

    return pd.DataFrame(rows)

In [28]:
results_dir = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/"

In [29]:
target_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/target_uids.csv")

In [30]:
def uid_to_item_store(uid: str):
    parts = str(uid).split('_')
    item_id  = '_'.join(parts[:-2])
    store_id = parts[-2] + '_' + parts[-1]
    return item_id, store_id

keys_from_target = list({
    uid_to_item_store(uid)
    for uid in target_df['unique_id'].dropna().astype(str)
})

In [31]:
meta_path = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/results_state_dept_3y_train/cv_meta.pkl"
with open(meta_path, "rb") as f:
    meta = pickle.load(f)

In [33]:
smooth_series = [
    "FOODS_2_152_WI_1",
    "FOODS_2_384_TX_3",
    "FOODS_3_080_WI_3",
    "FOODS_3_586_CA_1",
    "HOUSEHOLD_1_440_CA_3",
]


lumpy_series = [
    "FOODS_1_145_WI_3",
    "FOODS_2_222_WI_1",
    "FOODS_2_324_CA_1",
    "FOODS_3_454_CA_4",
    "HOUSEHOLD_2_371_WI_3",
]


erratic_series = [
    "FOODS_2_360_CA_3",
    "FOODS_3_339_CA_3",
    "FOODS_3_468_CA_3",
    "FOODS_3_476_CA_1",
    "HOUSEHOLD_1_506_TX_2",
]


intermittent_series = [
    "FOODS_1_177_TX_1",
    "FOODS_1_219_CA_3",
    "FOODS_3_031_TX_1",
    "FOODS_3_361_CA_4",
    "HOBBIES_1_217_WI_3",
]


In [34]:
# Check original 20 items
base_20_uids = (
    smooth_series
    + lumpy_series
    + erratic_series
    + intermittent_series
)

base_20_set = set(base_20_uids)

df_eval_20only = (
    df_eval_target[df_eval_target['SeriesName'].isin(base_20_set)]
    .copy()
)

group_map = {u: 'smooth' for u in smooth_series}
group_map.update({u: 'lumpy' for u in lumpy_series})
group_map.update({u: 'erratic' for u in erratic_series})
group_map.update({u: 'intermittent' for u in intermittent_series})

df_eval_20only['Group'] = df_eval_20only['SeriesName'].map(group_map)

display(df_eval_20only.sort_values(['Group','SeriesName']))

# summary
print("Counts:", df_eval_20only.shape[0], "/ expected 20")
display(
    df_eval_20only.groupby('Group', dropna=False)[['RMSE','Norm_RMSE','ME','Norm_ME','RMSSE']].mean().round(4)
)
display(
    df_eval_20only[['RMSE','Norm_RMSE','ME','Norm_ME','RMSSE']].mean().to_frame('Overall_mean').round(4).T
)

missing_20 = sorted(list(base_20_set - set(df_eval_20only['SeriesName'])))
if missing_20:
    print("Missing from eval:", missing_20)

,SeriesName,RMSE,Norm_RMSE,ME,Norm_ME,RMSSE,note,Group
77,FOODS_2_360_CA_3,17.488622,0.448683,-8.431824,-0.216324,0.868195,None,erratic
115,FOODS_3_339_CA_3,6.779724,0.470869,-0.465410,-0.032324,0.901569,None,erratic
102,FOODS_3_468_CA_3,4.864613,0.433672,0.201030,0.017921,0.519233,None,erratic
83,FOODS_3_476_CA_1,6.286108,0.367065,0.562108,0.032823,0.973728,None,erratic
63,HOUSEHOLD_1_506_TX_2,4.268456,0.471936,-0.075955,-0.008398,0.770802,None,erratic
122,FOODS_1_177_TX_1,1.486031,1.060607,-0.230209,-0.164304,1.377098,None,intermittent
82,FOODS_1_219_CA_3,5.255305,0.645893,-0.561123,-0.068964,1.086977,None,intermittent
39,FOODS_3_031_TX_1,1.730776,1.049575,-0.239836,-0.145441,1.299514,None,intermittent
59,FOODS_3_361_CA_4,1.701423,1.001330,0.060234,0.035449,1.192191,None,intermittent
119,HOBBIES_1_217_WI_3,0.678044,2.386449,-0.058081,-0.204423,2.049092,None,intermittent


Counts: 20 / expected 20


,RMSE,Norm_RMSE,ME,Norm_ME,RMSSE
Group,,,,,
erratic,7.9375,0.4384,-1.6420,-0.0413,0.8067
intermittent,2.1703,1.2288,-0.2058,-0.1095,1.4010
lumpy,4.9519,1.0732,-0.6290,-0.0244,1.2428
smooth,6.3251,0.3147,-0.4242,-0.0459,1.0109


,RMSE,Norm_RMSE,ME,Norm_ME,RMSSE
Overall_mean,5.3462,0.7638,-0.7253,-0.0553,1.1153
